In [ ]:
# load packages

import random
from glob import glob
import matplotlib.pyplot as plt
from matplotlib import colors, cm, transforms
import matplotlib.patches as patches
import numpy as np
import xarray as xr
from netCDF4 import Dataset
import math as math
import matplotlib.animation as animation
import matplotlib.patches as patches
from collections import Counter 
import pandas as pd
import datetime
import pickle
import xarray as xr
from scipy.stats import pearsonr
from scipy.interpolate import interp1d
from statsmodels.nonparametric.smoothers_lowess import lowess

import cartopy.crs as ccrs
import cartopy
import cartopy.feature as cfeature
from cartopy.io import shapereader
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
%matplotlib inline

## Analysis minimum time afloat

In [ ]:
# Load all parameters

# load trajectories

traj_path = '../particle_simulation/output/particles_epibionts_obs_30d.zarr'
Traj = xr.open_zarr(traj_path)
plon = np.array(Traj['lon'])
plat = np.array(Traj['lat'])
pocean = np.array(Traj['openocean_flag'])
dist_traveled = np.array(Traj['distance_traveled'])
dist2release = np.array(Traj['distance_to_release'])
dist2coast = np.array(Traj['distance_to_coast'])
ptime = np.array(Traj['time'])
outputfreq = 0.5 #days
pocean[dist2coast==0]=0 # correct pocean to 0 if dist2coast = 0

# load landmask, lon, lat, and startlon startlat

file = open('../particle_simulation/input/inputfiles_epibionts_full', 'rb')
inputfiles = pickle.load(file)
file.close()
landmask = inputfiles['landmask']
lon = inputfiles['lon']
lat = inputfiles['lat']
coastlon = inputfiles['lonSA'][:]
coastlat = inputfiles['latSA'][:]
file = open('../particle_simulation/input/inputfiles_epibionts', 'rb')
inputfiles = pickle.load(file)
file.close()
startlon = inputfiles['releaselon'][:]
startlat = inputfiles['releaselat'][:]   
pID = inputfiles['releaseID'][:]

# make coastline mask (specified at T-point)
coastmask = np.zeros_like(landmask)
for i in range(coastmask.shape[1]):
    for j in range(coastmask.shape[0]):
        if landmask[j,i]==1:
            coastmask[j-1:j+2,i-1:i+2]=1
coastmask = coastmask + landmask

# load observation estimates

fieldobs = '../particle_simulation/input/Data_Observations.xls'
obs = pd.read_excel(fieldobs) 
lat_obs = np.array(obs.Latitude)
lon_obs = np.array(obs.Longitude)
est_obs = np.array(obs.Estimate)
err_obs = np.array(obs.Label)

In [ ]:
# Calculate days afloat - 50d backtracking

dt = 1 # how big are the bins (in days) of 'time afloat'

# time afloat for each particle
time_period = 50 #how many days are we looking back in time
time_in_ocean = np.nansum(pocean[:,0:int(time_period/outputfreq)],axis=1)*outputfreq #outcome is in days

# quality check every particle (soms are stuck due to design)
# code determines how many particles are not stuck due to design before release
# at each location.
ptotal_loc = []
pquality = np.zeros_like(time_in_ocean)
for oloc in range(obs.shape[0]):
    teller = int(len(time_in_ocean)/len(lat_obs)) #start with all particles
    pselect = np.where(pID==oloc)[0][:]
    for p in pselect:
        if plat[p,0] == plat[p,-1]:
            teller -= 1
            pquality[p] = 0 #bad
        else:
            pquality[p] = 1 #good
    ptotal_loc.append(teller)

# bin results for measurement site x days afloat
#ptotal = int(len(time_in_ocean)/len(lat_obs)) # total number of particles per release location
days_afloat = np.arange(0,time_period+dt,dt) # lower limit of bins
distribution_afloat50 = np.zeros((len(lat_obs),len(days_afloat)-1))
distribution_afloat50cumsum = np.zeros((len(lat_obs),len(days_afloat)-1))

for i in range(len(lat_obs)):
    ptotal = ptotal_loc[i]
    for b in range(len(days_afloat)-1):
        check = ((time_in_ocean[pID==i]>=days_afloat[b]) & 
                (time_in_ocean[pID==i]<days_afloat[b+1]) &
                (pquality[pID==i]==1))
        distribution_afloat50[i,b] = np.sum(check)/ptotal*100

for i in range(len(lat_obs)):
    ptotal = ptotal_loc[i]
    for b in range(len(days_afloat)-1):
        check = ((time_in_ocean[pID==i]>=days_afloat[b]) &
                (pquality[pID==i]==1))
        distribution_afloat50cumsum[i,b] = np.sum(check)/ptotal*100

In [ ]:
# Save separate estimates to excel sheet

all_estimates = {}

all_estimates['Country'] = obs.Country[obs.Label=='confident'] 
all_estimates['Beach'] = obs.Beach[obs.Label=='confident']
all_estimates['Latitude'] = obs.Latitude[obs.Label=='confident']
all_estimates['Longitude'] = obs.Longitude[obs.Label=='confident']

for b in np.arange(1,len(days_afloat)-1):
    all_estimates[str(b)] = distribution_afloat50cumsum[obs.Label=='confident',b]

all_estimates = pd.DataFrame.from_dict(all_estimates)
    
all_estimates.to_excel('Model_all_estimates_50backward_new.xlsx')

## Figure per observation location

In [ ]:
# ZOOM IN example trajectories for all 44 observation locations with distribution of days afloat 

# time afloat for each particle
bins = np.arange(0,52,2)

# coastmask
coastmask = np.zeros_like(landmask)
for i in range(coastmask.shape[1]):
    for j in range(coastmask.shape[0]):
        if landmask[j,i]==1:
            coastmask[j-1:j+2,i-1:i+2]=1
coastmask = coastmask + landmask

#fix colors
coastmask[coastmask==0]=3
coastmask[coastmask==1]=6
coastmask[coastmask==2]=18

for oloc in range(obs.shape[0]):

    print(oloc)
    fig,ax2 = plt.subplots(1,2, figsize=(12,6))

    # plot distribution of time afloat
    ax2[1].bar(np.arange(0,50),distribution_afloat50[oloc,:])
    ax2[1].set_xlabel('days afloat')
    ax2[1].set_ylabel('percentage of particles')
    ax2[1].set_xlim([0,50])
    ax2[1].set_ylim([0,max(distribution_afloat50[oloc,:])+3])
    ax2[1].grid(True)

    # plot particles in map
    ax2[0].pcolor(lon[600,:],lat[:,300],coastmask,shading='nearest',
              cmap='tab20c',vmin=0, vmax=19,
              alpha=0.9)

    # plot measurement location
    ax2[0].scatter(obs.Longitude[oloc],obs.Latitude[oloc],s=90,c='red',alpha=0.8,zorder=2000)
    ax2[0].set_xlim([obs.Longitude[oloc]-3,obs.Longitude[oloc]+3])
    ax2[0].set_ylim([obs.Latitude[oloc]-3,obs.Latitude[oloc]+3])

    selection_particles = np.where(pID==oloc)[0][:]
    particles2plot = np.random.randint(selection_particles.shape[0], size=100)
    partplot = selection_particles[particles2plot]

    for p in partplot:
        ax2[0].plot(plon[p,:],plat[p,:],c='k',linewidth=0.1)

    cmap = plt.cm.get_cmap('tab20c')
    colors = np.linspace(0,1,num=20,endpoint=True)
    ax2[0].scatter(10,10,color=cmap(colors[3]),label='ocean')
    ax2[0].scatter(10,10,color=cmap(colors[6]),label='coast')
    ax2[0].scatter(10,10,color=cmap(colors[18]),label='land')
    ax2[0].legend()

    ax2[0].set_title('Zoom in on ' + obs.Beach[oloc] + ', ' + obs.Country[oloc])
    ax2[0].set_xlabel('Longitude ($^{\circ}$E)')
    ax2[0].set_ylabel('Latitude ($^{\circ}$N)')

    # Save figure
    plt.tight_layout()
    plt.savefig('Figures/zoomin_' + str(obs.Beach[oloc]).replace(" ", "_") + '_' + str(obs.Country[oloc]).replace(" ", "_") + '.png',
                dpi=300,facecolor='#ffffff')

## Figure distribution of minimum time afloat

In [ ]:
# Figure for showing distribution - 50d backtracking

distribution_afloat_conf = distribution_afloat50cumsum[obs.Label=='confident']
lat_obs_conf = obs.Latitude[obs.Label=='confident']
labels = np.arange(0,len(lat_obs_conf))+1
increments = np.arange(0,len(days_afloat)-1)

fig = plt.figure(figsize=(7,9))
gs = fig.add_gridspec(2, 1, width_ratios=[1], height_ratios = [1,0.05])

# plot model results
ax1 = fig.add_subplot(gs[0,0])
im=ax1.pcolormesh(increments, labels, distribution_afloat_conf[np.argsort(lat_obs_conf),:], cmap='ocean_r',vmin=0,vmax=100)

ax1.set_ylabel('Numbered measurement locations ordered by latitude (South to North)')
ax1.set_xlabel('Minimum time afloat (days)')
ax1.set_title('Distribution per measurement location')
ax1.set_xlim([0,max(days_afloat)])
ax1.set_ylim([0,40])

# add colorbar
#im = ax1.scatter(-10*(colors+10),-10*(colors+10),s=20,c=colors,cmap='ocean_r',vmin=0,vmax=max(days_afloat))
ax2 = fig.add_subplot(gs[1,0])
cbr = plt.colorbar(im, cax=ax2, orientation='horizontal')
cbr.set_label('Percentage of particles', fontsize=12)
cbr.ax.tick_params(labelsize=12)

# Save figure
plt.tight_layout()
plt.savefig('Figures/Sensitivity_daysafloat_30dmeasurement_50dbackward.png',dpi=300,facecolor='#ffffff')


## Figure model - observations comparison

In [ ]:
# Plot showing example trajectories towards each measurement location and correlation 

# Deriving interpolated and smoothened mainland variation - 50d backward
selection = np.array(obs[(obs.Label=='confident')].index)
corr_idx = 45

X = lat_obs[selection]
Xi = np.linspace(np.min(X), np.max(X), 76)
Ym = distribution_afloat50cumsum[selection,corr_idx]
Yo = est_obs[selection]

intmodel_model = interp1d(X[np.argsort(X)],Ym[np.argsort(X)], kind='linear')
intmodel_obs = interp1d(X[np.argsort(X)],Yo[np.argsort(X)], kind='linear')

Yi_m = intmodel_model(Xi)
Yi_o = intmodel_obs(Xi)

N=3
Ys_m = np.convolve(Yi_m, np.ones(N)/N, mode='valid')
Ys_o = np.convolve(Yi_o, np.ones(N)/N, mode='valid')

# start figure

fig = plt.figure(figsize=(13,13))
gs = fig.add_gridspec(1, 2, width_ratios=[1, 1], height_ratios = [1])

# add figure map
ax1 = fig.add_subplot(gs[:,0])

landmask[landmask==0]=3
landmask[landmask==1]=18
ax1.pcolor(lon[600,:],lat[:,300],landmask,shading='nearest',
           cmap='tab20c',vmin=0, vmax=19, alpha=0.9)

# Plot particle trajectories
release_loc = np.array(obs[(obs.Label=='confident')].index)
p_indx = np.arange(0,plon.shape[0])

for locs in release_loc:
    particles2plot = random.sample(list(p_indx[pID==locs]),25)
    for p in particles2plot:
        ax1.plot(plon[p,0:100],plat[p,0:100],c='k',linewidth=0.1)

# scatter measurements
for measurement in range(len(obs)):
    if obs.Label[measurement] == 'confident':
        ax1.scatter(lon_obs[measurement],lat_obs[measurement],s=(est_obs[measurement]+1)*30,color='#b45f06',alpha=0.5,zorder=2000)
        
ax1.scatter(0,0,s=30,color='#b45f06',alpha=0.5,label=' 1%')        
ax1.scatter(0,0,s=150,color='#b45f06',alpha=0.5,label=' 5% observed items from ocean estimate')  
ax1.scatter(0,0,s=300,color='#b45f06',alpha=0.5,label='10%')  
ax1.plot([0,1],[0,1],c='k',linewidth=1,label='50-day backwards example trajectories')

ax1.legend(prop={"size":11}, ncol=1, loc='upper right')
ax1.set_xlim([-120,-69])
ax1.set_ylim([-50,34])
ax1.set_xlabel('Longitude ($^{\circ}$E)')
ax1.set_ylabel('Latitude ($^{\circ}$N)')

## Add second panel with resulting correlation
ax2 = fig.add_subplot(gs[:,1])
ax3 = ax2.twiny()
omax=20 #xlimit for observations
mmax=101 #xlimit for model

for oloc in range(distribution_afloat50cumsum.shape[0]):
    
    odata = obs.Estimate[oloc]
    mdata = distribution_afloat50cumsum[oloc,corr_idx]
    
    if obs.Label[oloc] == 'confident':
        ax3.scatter(distribution_afloat50cumsum[oloc,corr_idx],lat_obs[oloc],s=50,color='k',marker=5,alpha=0.5,zorder=200)
        ax2.scatter(obs.Estimate[oloc],lat_obs[oloc],s=30,color='#b45f06',alpha=1,zorder=300)
        if mmax/omax*odata > mdata:
            ax3.plot([mdata,mmax/omax*odata],[lat_obs[oloc],lat_obs[oloc]],color='k',linewidth=0.3)
        else:
            ax3.plot([mmax/omax*odata,mdata],[lat_obs[oloc],lat_obs[oloc]],color='k',linewidth=0.3)

#get legends
ax2.scatter(10,10,s=30,color='#b45f06',label='observed estimate')
ax2.scatter(10,10,s=50,color='k',marker=5,alpha=0.5,label='model estimate')

#axis settings
ax2.set_ylim([-50,34])
ax2.set_xlim([0,omax])
ax3.set_xlim([0,mmax])

ax2.set_yticklabels([])
ax2.set_xlabel('Observed items from ocean estimate (%)')     
ax3.set_xlabel('Modeled particles in open ocean longer than ' + str(corr_idx) + ' days (%)')
ax2.legend(loc='upper right',prop={"size":12})

gs.update(wspace=0.05)

# add smooth curve in panel (b)
ax3.plot(Ys_m,Xi[int(N/2):-int(N/2)],color='grey')
ax3.fill_betweenx(Xi[int(N/2):-int(N/2)],0,Ys_m,color='grey',alpha=0.2)
ax2.plot(Ys_o,Xi[int(N/2):-int(N/2)],color='#b45f06')
ax2.fill_betweenx(Xi[int(N/2):-int(N/2)],0,Ys_o,color='#b45f06',alpha=0.2)

# add figure labels
ax1.text(0.01, 0.01, 'a)', transform=ax1.transAxes, fontsize=15,
        verticalalignment='bottom', horizontalalignment='left')
ax2.text(0.01, 0.01, 'b)', transform=ax2.transAxes, fontsize=15,
        verticalalignment='bottom', horizontalalignment='left')

# Save figure
plt.tight_layout()
plt.savefig('Figures/map_correlation_30dmeasurement_45minimumafloat.png',dpi=300,facecolor='#ffffff')

## Figure with selection of undrogued, grounded GDP data

Selection along the East Pacific Coastline.

In [ ]:
# Figure with selection of GDP data

GDPdata = pd.read_excel('EPC.xlsx')

# plot map
plt.figure(figsize=(10,10))
ax1 = plt.subplot(211, projection=ccrs.PlateCarree(central_longitude=180))
ax1.set_global()
#ax1.coastlines(resolution='50m', color='grey', linewidth=0.2)
#ax1.add_feature(cartopy.feature.BORDERS, color='grey',linewidth=0.2)
#ax1.add_feature(cartopy.feature.LAND)
#ax1.add_feature(cartopy.feature.OCEAN,facecolor='#C6EBF1')
ax1.set_xticks([0, 60, 120, 180, 240, 300, 360], crs=ccrs.PlateCarree())
ax1.set_yticks([-90, -60, -30, 0, 30, 60, 90], crs=ccrs.PlateCarree())
axextent = [-250, -50, -65, 65]
ax1.set_extent(axextent, crs=cartopy.crs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

# plot  GDP data
ax1.scatter(GDPdata.LONG-180,GDPdata.LAT,s=1,c='black')

# save figure
plt.tight_layout()
plt.savefig('figures/Map_GDPgrounding_EastPacific_onlydrifters.png',dpi=300, facecolor='#FFFFFF')